### Assignment 2 Part B

#### Implement a neural network classifier for the loan data with Decision as the output attribute. 
Prepare the data as needed. Come up with your best performing model by changing the size and number of hidden layers and activation functions.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold




2025-05-20 23:41:29.851381: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747798889.875892   34974 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747798889.883401   34974 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747798889.902615   34974 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747798889.902637   34974 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747798889.902639   34974 computation_placer.cc:177] computation placer alr

In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Load data
data = pd.read_excel('loan.xlsx')
data = data.dropna().drop_duplicates()

# Encode target (Decision)
data['Decision'] = data['Decision'].map({'accept': 1, 'reject': 0})

# Remove outliers using IQR method
def remove_outliers(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[column] > lower_bound) & (data[column] < upper_bound)]

numerical_cols = ['Age', 'Time_at_address', 'Time_employed', 'Time_bank', 'Home_Expn', 'Balance']
for col in numerical_cols:
    data = remove_outliers(data, col)

# Separate features and target
X = data.drop('Decision', axis=1)
y = pd.get_dummies(data['Decision'])

# Categorical columns
categorical_cols = ['Sex', 'Res_status', 'Telephone', 'Occupation', 'Job_status', 'Liab_ref', 'Acc_ref']

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

# Transform features
X_encoded = preprocessor.fit_transform(X)

# Get feature names
feature_names = (numerical_cols +
                 preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_cols).tolist())
X_encoded_df = pd.DataFrame(X_encoded.toarray() if hasattr(X_encoded, "toarray") else X_encoded,
                            columns=feature_names)

# Define K-Fold Cross-Validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Model configurations to test
hidden_layer_options = [
    [64, 32],
    [128, 64, 32],
    [32],
    [64, 64, 64]
]
activation_functions = ['relu', 'tanh', 'elu']

# Store results
results = []

# Perform K-fold cross-validation and search for the best architecture
for layers in hidden_layer_options:
    for activation in activation_functions:
        print(f"Testing architecture: {layers} with activation: {activation}")
        
        fold_scores = []
        
        # Perform K-fold cross-validation
        for fold, (train_index, val_index) in enumerate(kf.split(X_encoded_df), 1):
            print(f"Fold {fold}")
            
            # Split data into training and validation based on the fold
            X_train, X_val = X_encoded_df.iloc[train_index], X_encoded_df.iloc[val_index]
            y_train, y_val = y.iloc[train_index], y.iloc[val_index]

            # Build model dynamically
            model = Sequential()
            model.add(Dense(layers[0], activation=activation, input_shape=(X_train.shape[1],)))
            for units in layers[1:]:
                model.add(Dense(units, activation=activation))
                model.add(BatchNormalization())
            model.add(Dropout(0.2))
            model.add(Dense(2, activation='softmax'))

            model.compile(optimizer=Adam(learning_rate=0.0001),
                          loss='categorical_crossentropy',
                          metrics=['accuracy'])

            # Early stopping callback
            early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

            # Train model on this fold
            history = model.fit(X_train, y_train,
                                validation_data=(X_val, y_val),
                                epochs=175,
                                batch_size=10,
                                verbose=0,
                                callbacks=[early_stop])

            # Evaluate model performance on the validation set of the fold
            _, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
            fold_scores.append(val_accuracy)
            print(f"Fold {fold} Accuracy: {val_accuracy:.4f}")
        
        # Calculate mean accuracy for the current architecture across all folds
        mean_accuracy = np.mean(fold_scores)
        print(f"Mean Accuracy for architecture {layers} with activation {activation}: {mean_accuracy:.4f}\n")
        
        # Store results
        results.append({
            'layers': layers,
            'activation': activation,
            'accuracy': mean_accuracy
        })

# Find best model based on mean cross-validation accuracy
best_model = max(results, key=lambda x: x['accuracy'])

print("\n🔍 Best Model Configuration (Cross-Validation):")
print(f"Layers: {best_model['layers']}")
print(f"Activation: {best_model['activation']}")
print(f"Mean Accuracy: {best_model['accuracy']:.4f}")


Testing architecture: [64, 32] with activation: relu
Fold 1


/home/a.hasmukhkumarjain/.conda/envs/csye7380/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Fold 1 Accuracy: 0.7200
Fold 2


/home/a.hasmukhkumarjain/.conda/envs/csye7380/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Fold 2 Accuracy: 0.5800
Fold 3


...

Fold 3 Accuracy: 0.7755
Fold 4


/home/a.hasmukhkumarjain/.conda/envs/csye7380/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Fold 4 Accuracy: 0.5918
Fold 5


/home/a.hasmukhkumarjain/.conda/envs/csye7380/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Fold 5 Accuracy: 0.7551
Mean Accuracy for architecture [64, 64, 64] with activation elu: 0.7205


🔍 Best Model Configuration (Cross-Validation):
Layers: [32]
Activation: elu
Mean Accuracy: 0.7696
